In [53]:
import os
import io
import s3fs
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import seaborn as sns

!pip install missingno
import missingno as msno

!pip install nltk
import nltk as nltk

%matplotlib inline

#fichier projet
from bdd import *
from preprocessing import * 

Looking in indexes: https://nexus.beta.innovation.insee.eu/repository/pypi-public/simple
Looking in indexes: https://nexus.beta.innovation.insee.eu/repository/pypi-public/simple


In [45]:
my_driver = PostGre_SQL_DB()

Toutes les reqûetes sont testé sur les dataset de 2014/2017/2018/2019

In [49]:
df = my_driver.read_from_sql(''' 
SELECT count(*) as count
FROM rp_final_2017
WHERE 1=1
AND rs_x = ''
OR rs_x IS NULL; ''')
df.head()

,count
0,19882


In [50]:
df = my_driver.read_from_sql(''' 
SELECT count(*) as count
FROM rp_final_2018
WHERE 1=1
AND rs_x = ''
OR rs_x IS NULL; ''')
df.head()

,count
0,19958


In [51]:
df = my_driver.read_from_sql(''' 
SELECT count(*) as count
FROM rp_final_2019
WHERE 1=1
AND rs_x = ''
OR rs_x IS NULL; ''')
df.head()

,count
0,20095


Combien de siret codé par les gestionnaire sont écraser par R3 lors de l'arbitrage ?

In [10]:
df = my_driver.read_from_sql(''' SELECT count(*)
FROM rp_final_2017 a
WHERE a.siretm IS NOT NULL
AND a.siretm != a.siret_dec; ''')

In [11]:
df.head()

,count
0,0


Isolons les BI qui n'ont pas été codé par MCA ni par un gestionnaire

In [12]:
df = my_driver.read_from_sql(''' 
SELECT *
FROM rp_final_2019
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
LIMIT 10; ''')

In [15]:
df.head()

,cabbi,typcol,depcom_code,rs_x,ilt_x,vardompart_x,numvoi_x,bister_x,typevoi_x,nomvoi_x,cpladr_x,actet_x,actet_c_c,actet_c,clt_c_c,clt_x,dlt_x,plt_x,siretc,note_mca_adr_c,note_mca_nom_c,i_mca_c,profi_x,profs_x,profa_x,i_reprise_act,i_siret_m,siret_dec,siretm,proces_codage_init,arbitrage,siretq,i_siret_q,siret_arb,x,y,sourcexyw,qual,siret_final,code_retour,naf1,pct1,naf2,pct2,naf3,pct3,naf4,pct4,naf5,pct5,nbtot,nb5
0,9350454078,1.0,44165,ADMR,1,1,None,None,None,None,None,AIDE A DOMICILE,None,8810A,44165,None,None,None,None,None,None,None,None,AIDE A DOMICILE,None,V,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,9350453758,1.0,44165,CONSUEL,2,1,None,None,None,None,None,ORGANISME DE CONTROLE,None,7120B,44109,NANTES,44,None,None,None,None,None,None,INSPECTEUR DE CHANTIER,None,V,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,9350441305,1.0,33056,MINISTERE DE LA DEFENSE,2,None,None,None,AV,DE L'ARGONNE,None,SECURITE DE L'ETAT,None,8422Z,33281,MERIGNAC,33,None,None,None,None,None,None,TECHINCIEN CATEGORIE B,None,V,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,9350454689,1.0,44165,H2O ATHOME,None,2,None,None,None,None,None,VENTE A DOMICILE DE PRODUITS ECOLOGIQUES POUR ...,None,4799A,44165,None,None,None,None,None,None,None,CONSEILLERE DE VENTE A DOMICILE,None,None,V,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,9350454731,1.0,44165,FAMILLES RURALES,2,None,0007,None,R,LES 2 RIVES,None,ACCUEIL DE LOISIRS,None,9499Z,85217,SAINT-GEORGES-DE-MONTAIGU,85,None,None,None,None,None,None,DIRECTRICE D ACCUEIL DE LOISIRS,None,V,0.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


-> La ligne n°3 a pour rs_x 'H2O ATHOME', si on regarde sur Internet il n'y a pas d'occurence. Mais si on écrit 'H2O AT HOME', on retrouve une boutique https://www.h2oathome.fr/nettoyage-ecologique qui semble coller à la profession. L'erreur a du échapper au gestionnaire.   
=> Codage imparfait

Combien de code non trouvé ?

In [20]:
df = my_driver.read_from_sql(''' 
SELECT COUNT(*)
FROM rp_final_2014
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
; ''')
df.head()

,count
0,0


In [21]:
df = my_driver.read_from_sql(''' 
SELECT COUNT(*)
FROM rp_final_2017
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
; ''')
df.head()

,count
0,125855


In [22]:
df = my_driver.read_from_sql(''' 
SELECT COUNT(*)
FROM rp_final_2018
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
; ''')
df.head()

,count
0,111838


In [23]:
df = my_driver.read_from_sql(''' 
SELECT COUNT(*)
FROM rp_final_2019
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
; ''')
df.head()

,count
0,111870


Quels sont les rs_x les plus difficile à coder?

In [26]:
df = my_driver.read_from_sql(''' 
SELECT rs_x, count(rs_x) as count
FROM rp_final_2017
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY rs_x
ORDER BY count DESC
; ''')
df.head(15)

,rs_x,count
0,EDUCATION NATIONALE,2143
1,MINISTERE DE LA DEFENSE,1737
2,ASSISTANTE MATERNELLE,1617
3,CESU,1580
4,DIVERS,1167
5,PARTICULIERS,988
6,PARTICULIER,950
7,GENDARMERIE,895
8,ADECCO,733
9,GENDARMERIE NATIONALE,715


ADMR (signifiant Aide à domicile en milieu rural)  
Adecco groupe suisse spécialisé dans l'intérim  
CESU Centre d'enseignement des soins d'urgence ?  
Manpower gros cabinet de conseil, qui contient de l'interim

In [27]:
df = my_driver.read_from_sql(''' 
SELECT rs_x, count(rs_x) as count
FROM rp_final_2018
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY rs_x
ORDER BY count DESC
; ''')
df.head(15)

,rs_x,count
0,EDUCATION NATIONALE,2136
1,CESU,1805
2,ASSISTANTE MATERNELLE,1650
3,DIVERS,1073
4,MINISTERE DE LA DEFENSE,1038
5,GENDARMERIE,915
6,PARTICULIERS,871
7,PARTICULIER,839
8,MINISTERE DES ARMEES,762
9,ADECCO,760


In [28]:
df = my_driver.read_from_sql(''' 
SELECT rs_x, count(rs_x) as count
FROM rp_final_2019
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY rs_x
ORDER BY count DESC
; ''')
df.head(15)

,rs_x,count
0,EDUCATION NATIONALE,2365
1,CESU,1969
2,ASSISTANTE MATERNELLE,1497
3,DIVERS,1083
4,GENDARMERIE,948
5,MINISTERE DES ARMEES,908
6,PARTICULIERS,901
7,PARTICULIER,897
8,MINISTERE DE LA DEFENSE,836
9,GENDARMERIE NATIONALE,751


Profession difficile à coder ?

In [39]:
df = my_driver.read_from_sql(''' 
SELECT PROFS_X, count(PROFS_X) as count
FROM rp_final_2017
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY PROFS_X
ORDER BY count DESC
; ''')
df.head(15)

,profs_x,count
0,ASSISTANTE MATERNELLE,7042
1,AGENT D ENTRETIEN,2236
2,AIDE A DOMICILE,1715
3,PROFESSEUR DES ECOLES,1366
4,FEMME DE MENAGE,1252
5,MACON,1169
6,MILITAIRE,1135
7,AUXILIAIRE DE VIE,952
8,AGENT D'ENTRETIEN,881
9,GENDARME,851


In [40]:
df = my_driver.read_from_sql(''' 
SELECT PROFS_X, count(PROFS_X) as count
FROM rp_final_2018
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY PROFS_X
ORDER BY count DESC
; ''')
df.head(15)

,profs_x,count
0,ASSISTANTE MATERNELLE,6682
1,AGENT D ENTRETIEN,1771
2,AIDE A DOMICILE,1557
3,PROFESSEUR DES ECOLES,1371
4,MILITAIRE,1277
5,FEMME DE MENAGE,1102
6,MACON,947
7,GENDARME,870
8,AUXILIAIRE DE VIE,868
9,AGENT D'ENTRETIEN,861


In [41]:
df = my_driver.read_from_sql(''' 
SELECT PROFS_X, count(PROFS_X) as count
FROM rp_final_2019
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY PROFS_X
ORDER BY count DESC
; ''')
df.head(15)

,profs_x,count
0,ASSISTANTE MATERNELLE,6426
1,AGENT D ENTRETIEN,1718
2,AIDE A DOMICILE,1625
3,PROFESSEUR DES ECOLES,1422
4,MILITAIRE,1418
5,FEMME DE MENAGE,1139
6,MACON,1101
7,AGENT D'ENTRETIEN,878
8,AUXILIAIRE DE VIE,870
9,GENDARME,861


In [42]:
df = my_driver.read_from_sql(''' 
SELECT PROFI_X, count(PROFI_X) as count
FROM rp_final_2017
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY PROFI_X
ORDER BY count DESC
; ''')
df.head(15)

,profi_x,count
0,ASSISTANTE MATERNELLE,901
1,AGRICULTEUR,616
2,COMMERCANT,314
3,INFIRMIERE LIBERALE,295
4,INFIRMIERE,242
5,ARTISAN,240
6,MACON,214
7,AIDE A DOMICILE,206
8,GERANT,205
9,FEMME DE MENAGE,204


In [43]:
df = my_driver.read_from_sql(''' 
SELECT PROFI_X, count(PROFI_X) as count
FROM rp_final_2018
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY PROFI_X
ORDER BY count DESC
; ''')
df.head(15)

,profi_x,count
0,ASSISTANTE MATERNELLE,812
1,AGRICULTEUR,365
2,COMMERCANT,286
3,AIDE A DOMICILE,218
4,INFIRMIERE LIBERALE,213
5,FEMME DE MENAGE,204
6,MACON,193
7,ARTISAN,185
8,GERANT,179
9,INFIRMIERE,173


In [44]:
df = my_driver.read_from_sql(''' 
SELECT PROFI_X, count(PROFI_X) as count
FROM rp_final_2019
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY PROFI_X
ORDER BY count DESC
; ''')
df.head(15)

,profi_x,count
0,ASSISTANTE MATERNELLE,708
1,AGRICULTEUR,389
2,COMMERCANT,249
3,AIDE A DOMICILE,243
4,ARTISAN,185
5,FEMME DE MENAGE,180
6,INFIRMIERE LIBERALE,179
7,MACON,175
8,GERANT,163
9,INFIRMIERE,161


En regardant la lognue traine du résultat, quelques résultats sont de type 'XSHQUXHSUIQ'. Ce type de 'grief' sont probablement de BI en ligne. Quelle est la proportion de oline/offline de siret non code ?  
typcol 1 -> internet  
typcol 0 -> papier

In [31]:
df = my_driver.read_from_sql(''' 
SELECT  typcol, count(*) as total_cat
FROM rp_final_2018
GROUP BY typcol
; ''')
df.head(15)

,typcol,total_cat
0,0.0,583669
1,1.0,1056351


In [32]:
df = my_driver.read_from_sql(''' 
SELECT  typcol, count(*) as total_cat
FROM rp_final_2019
GROUP BY typcol
; ''')
df.head(15)

,typcol,total_cat
0,0.0,532442
1,1.0,1127224


In [35]:
df = my_driver.read_from_sql(''' 
SELECT typcol, count(*) as count
FROM rp_final_2018
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY typcol
; ''')
df.head(15)

,typcol,count
0,0.0,54365
1,1.0,57473


In [37]:
df = my_driver.read_from_sql(''' 
SELECT typcol, count(*) as count
FROM rp_final_2019
WHERE 1=1
AND (siretc IS NULL
OR siretc = '')
AND (siretm	IS NULL
OR siretm = '')
GROUP BY typcol
; ''')
df.head(15)

,typcol,count
0,0.0,50861
1,1.0,61009


Le comptage en ligne est deux fois plus important que celui papier mais il conserve le meme taux de non-codé.

Qualité des champs libre:  
On commence par un champs simple, celui des commune, clt_x  
On va constituer un dictionnaire contenant tout les mots qui constitue les communes

In [60]:
df = my_driver.read_from_sql(''' 
SELECT DISTINCT(sir_adr_et_com_lib)
FROM sirus_2017
 ''')
communes_sirus = df.sir_adr_et_com_lib.values
df.head()

,sir_adr_et_com_lib
0,ETH
1,VERNAUX
2,LIZY
3,ESTERENCUBY
4,SAUMANE


In [61]:
communes_sirus_cleaned = [remove_punctuation(x) for x in communes_sirus]
communes_sirus_cleaned = " ".join(communes_sirus_cleaned)
mots_commune = nltk.word_tokenize(communes_sirus_cleaned)
mots_commune = list(filter(None, mots_commune))
unique_mot = list(set(mots_commune))

On prend tout les mots saisi dans clt_x

In [63]:
df = my_driver.read_from_sql(''' 
SELECT DISTINCT(clt_x)
FROM rp_final_2017
WHERE clt_x IS NOT NULL
 ''')
commune_saisie = df
df.head()

,clt_x
0,75009PARIS
1,SAUMANE
2,ESTERENCUBY
3,PIERREFITTE-NESTALAS
4,REGION PARISIENNEN ET NORMANDI


In [72]:
commune_saisies_string = " ".join(commune_saisie.clt_x.values)
token = nltk.word_tokenize(commune_saisies_string)
utoken = list(set(token))
token_cleaned = [x.strip() for x in utoken]
token_cleaned = list(filter(None, token_cleaned))

On compare les mots saisi et les mots du dictionnaire. Tout les mots qui ne sont pas trouvé ont donc un problème.

In [73]:
mismatch  = [x for x in token_cleaned if x not in unique_mot]

In [75]:
mismatch[0:25]

['54420',
 'CORNUAILLE',
 'OISSEAU-LE-PETIT',
 "MAS-D'ORCIERES",
 'GUEROULDE',
 'JUIGNE-SUR-LOIRE',
 'WASQHEHAL',
 'BRIGNAIS69',
 'VILLACOUBLAI',
 'BRETTEVILLE-DU-GRAND-CAUX',
 '69290',
 'VILEUFRANCHE',
 '59460',
 'PLACHY-BUYON',
 '97463',
 'JOL',
 'OTTROTT67',
 'PHILISBURG',
 'SAINT-MARTIN-LE-GREARD',
 'BLERIOT',
 'SAINT-PRIEST-LA-PRUGNE',
 '94805',
 'SAINT-EPAIN',
 'ARMANVILLE',
 'PETITE-FORET']

Dans les mots non trouvé:
- faute orthographe
    - Mauvaise orthographe
    - Lettre manquante (requete LIKE)
- ville frontalière
- nom de commune ayant disparu/fusionné/renommé
---
On peut voir aussi des communes avec des numéro aveur leur code postal accolé. Pour paris on peut voir les n° d'arrondissement en chiffre romains.  
On peut voir le code postal au lieu du nom.